In [1]:
# ===============  Importing Suitable Libraries ==========================
from tkinter import *                                                   
import tkinter as tk
from tkinter import ttk
import webbrowser
from PIL import ImageTk, Image
import googlesearch
from tkinter.filedialog import askopenfilename 
import numpy as np
from keras.models import load_model
import cv2
import pandas as pd
import keras
from pandastable import Table, TableModel
import tkinter.font as font
from keras.preprocessing.image import img_to_array
from tkinter import filedialog

# ================== .............. End..........============================

CONFIG

In [2]:
classes = { 0:'Cassava Bacterial Blight (CBB)',
 			1:'Cassava Brown Streak Disease (CBSD)', 
 			2:'Cassava Green Mottle (CGM)', 
 			3:'Cassava Mosaic Disease (CMD)', 
 			4:'Healthy',}

HELPER FUNCTION

In [3]:
#Helper function to open image
# Code from : https://github.com/MRK4863/Traffic-Sign-Classifier-with-GUI/blob/aef968461bde44db918567574ed48aef87a4c2bf/GUI_traffic_sign.py#L4
def openfilename():
    #progress = Progressbar(root, orient = HORIZONTAL, length =100, mode = 'determinate')
    filepath = askopenfilename()
    return filepath
def disp_image():
    print('Displaying Image.......')
    img=Image.open(fname)
    #resize image
    #code :https://github.com/iljared98/Tkinter-Image-Resizer/blob/master/main.pyw
    newImg = img.resize((350,200))
    photo = ImageTk.PhotoImage(newImg)
    labelphoto = Label(root, image = photo)
    labelphoto.image = photo
    labelphoto.place(x=100, y=150)
    # w = Canvas(root)
    # w.photo = photo
    # w.place(x=10 , y=120 )
def upload_img(): 
    global fname
    # Select the Imagename  from a folder  
    fname = openfilename() 
    disp_image()
    print(fname)
      
def callback(url):
        webbrowser.open(url) 
def browse_button():
    # Allow user to select a directory and store it in global var
    # called folder_path
    filename = filedialog.askdirectory()
    if filename != "":
        folder_path.set(filename)
    finished_statement.set("Click the button to start cleaning")

HELPER FUNCTION TO PREDICT IMG

In [4]:
import os
import sys
import hashlib
from collections import defaultdict
import time


def chunk_reader(fobj, chunk_size=1024):
    """ Generator that reads a file in chunks of bytes """
    while True:
        chunk = fobj.read(chunk_size)
        if not chunk:
            return
        yield chunk


def get_hash(filename, first_chunk_only=False, hash_algo=hashlib.sha1):
    hashobj = hash_algo()
    with open(filename, "rb") as f:
        if first_chunk_only:
            hashobj.update(f.read(1024))
        else:
            for chunk in chunk_reader(f):
                hashobj.update(chunk)
    return hashobj.digest()


def check_for_duplicates(paths, statement):

    z=0
    start_time = time.time()
    removed = 0
    skipped = 0

    files_by_size = defaultdict(list)
    files_by_small_hash = defaultdict(list)
    files_by_full_hash = dict()

    for path in paths:
        for dirpath, dirnames, filenames in os.walk(path):
            for filename in filenames:
                full_path = os.path.join(dirpath, filename)
                try:
                    # if the target is a symlink (soft one), this will
                    # dereference it - change the value to the actual target file
                    full_path = os.path.realpath(full_path)
                    file_size = os.path.getsize(full_path)
                except OSError:
                    # not accessible (permissions, etc) - pass on
                    continue
                files_by_size[file_size].append(full_path)
                print("Check " + str(z))
                z+=1

    # For all files with the same file size, get their hash on the first 1024 bytes
    for files in files_by_size.values():
        if len(files) < 2:
            continue  # this file size is unique, no need to spend cpu cycles on it

        for filename in files:
            try:
                small_hash = get_hash(filename, first_chunk_only=True)
            except OSError:
                # the file access might've changed till the exec point got here
                continue
            files_by_small_hash[small_hash].append(filename)
            print("Check " + str(z))
            z+=1

    
    # For all files with the hash on the first 1024 bytes, get their hash on the full
    # file - collisions will be duplicates
    for files in files_by_small_hash.values():
        if len(files) < 2:
            # the hash of the first 1k bytes is unique -> skip this file
            continue

        for filename in files:
            try:
                full_hash = get_hash(filename, first_chunk_only=False)
            except OSError:
                # the file access might've changed till the exec point got here
                continue

            if full_hash in files_by_full_hash:
                duplicate = files_by_full_hash[full_hash]
                print("Duplicate found:\n - %s\n - %s\n" % (filename, duplicate))
                filename = filename.replace('\\\\','\\')
                try:
                    os.remove(filename)
                except PermissionError:
                    skipped+=1
                    print("Check "+ str(z) +"skipped")
                removed +=1
            else:
                files_by_full_hash[full_hash] = filename
            print("Check " + str(z))
            z+=1

    statement.set("✅ Finished cleaning - "+ str(removed-skipped) + " files removed in "+ str(round(time.time() - start_time, 4)) + " seconds & "+str(skipped)+" files with permission error")

In [5]:
img_size=(300,300)
def model_predict(img_path):
    No_label='None'
    img = cv2.imread(img_path)
    image = cv2.resize(img, (img_size[0],img_size[1]))
    image = image.astype("float") / 255.0
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    model=load_model('mobinet_bhuy300x300.h5')
    proba=model.predict(image)
    idx = np.argmax(proba)
    if (idx==1 or idx==2 or idx==0 or idx==3 or idx==4):
        return idx
    else :
        return No_label
def predict_img():
    print('Entered successfully')
    print(fname)
    cid = model_predict(fname)
    
    global class_id
    class_id = cid
    print(classes[cid])
    
    #w = Label(root, text=classes[cid])
    #w.place( x= 200, y= 450)
    text = Text(root,height = 1, width = 30,
                font=('verdana',15,'bold'),fg="#248aa2")
    #text = Text(root, height = 1, width = 40)
    text.insert(INSERT, classes[class_id])
    text.place(x= 30, y= 490)

In [6]:
#creating main window
root =  tk.Toplevel()#tk.Tk()
#title of window
root.title("Dự đoán cây bệnh")
#window size
root.geometry("1100x600")
#window max-size/
root.maxsize(1100,600)
#window min-size
root.minsize(1100,600)
# #window icon
frame1 = LabelFrame(root,text="CASSAVA DISEASE",width=500,height=300,font=('verdana',15,'bold'),borderwidth=3,relief=RIDGE,highlightthickness=4,
                    bg="white",highlightcolor="white",highlightbackground="white",fg="#248aa2").place(x=30,y=90)
frame2 = LabelFrame(root,width=200,height=200,font=('verdana',15,'bold'),borderwidth=3,relief=RIDGE,highlightthickness=4,
                    bg="white",highlightcolor="white",highlightbackground="white",fg="#248aa2").place(x=550,y=89)
frame3 = LabelFrame(root,width=200,height=200,font=('verdana',15,'bold'),borderwidth=3,relief=RIDGE,highlightthickness=4,
                    bg="white",highlightcolor="white",highlightbackground="white",fg="#248aa2").place(x=800,y=89)
#search Button
# lucky = Button(root, text="i' m Felling Lucky",relief=RIDGE,font=('arial',10),bg="#F3F3F3",fg="#222222",cursor="hand2")
# lucky.place(x=400,y=360)
#-------------------------------Styling-----------------------------------#
# Styling
#Front nền 
titleFont = font.Font(size=18, weight=font.BOLD)
showLabelFont = font.Font(family="Helvetica")
finishedFont = font.Font(family="Helvetica", size=8)
browseButtonFont = font.Font(family="Arial", size=10, weight=font.BOLD)
deleteButtonFont = font.Font(family="Arial", size=12, weight=font.BOLD)

middle=0.9
primary = "#303444"
secondary = "#3E4458"
white = "#fff"
#---------------------Stringvar---------------#

folder_path = StringVar(value="Please select a folder")
finished_statement = StringVar(value="Click the button to start cleaning")

deleteButton = Button(text="CLEAN", command=lambda:check_for_duplicates([folder_path.get()], finished_statement),
                     bg=white, fg=primary, height=12, width=12)
deleteButton.place(relx=middle, rely=0.76, anchor=CENTER)
deleteButton['font'] = deleteButtonFont

#----------------------Label interface------------------------#

gd = Label(root,text="PREDICT CASSAVA DISEASE",font=('verdana',15,'bold'),
                    highlightcolor="white",fg="#FF0000")
gd.place(x=110,y=60)

apps = Label(root,text="More information ",relief=RIDGE,font=('arial',10),bg="#F3F3F3",fg="#222222",cursor="hand2")
apps.place(x=40,y=400) 
apps.bind("<Button-1>",lambda e: 
    callback("https://apps.lucidcentral.org/pppw_v10/text/web_full/entities/search.html?zoom_query=Cassava+disease"))

i = Label(root,text="PREDICT RESULT",font=('verdana',15,'bold'),
                    highlightcolor="white",fg="#FF0000")
i.place(x=30,y=450)

#Gmail label
gmail =  Label(root,text="Gmail",font=('verdana',9,'bold'),fg="black",cursor="hand2")
gmail.place(x=30,y=10)
gmail.bind("<Button-1>",lambda e: callback("https://mail.google.com/mail/"))

#facebook label
facebook =  Label(root,text="Facebook",font=('verdana',9,'bold'),fg="black",cursor="hand2")
facebook.place(x=80,y=10)
facebook.bind("<Button-1>",lambda e: callback("https://www.facebook.com/nguyen.bahuy.961"))

#Github label
Github=  Label(root,text="Github",font=('verdana',9,'bold'),fg="black",cursor="hand2")
Github.place(x=150,y=10)
Github.bind("<Button-1>",lambda e: callback("https://github.com/BaHuy15"))

# #-------------------------------Check duplicate-----------------------------------#
remove = Label(root,text="Realtime detection(not work)",font=('verdana',15,'bold'),
                    highlightcolor="white",fg="#FF0000")
remove.place(x=605,y=60) 

remove = Label(root,text="Remove duplicated image",font=('verdana',15,'bold'),
                    highlightcolor="white",fg="#FF0000")
remove.place(x=605,y=300) 

canvas1  = Canvas(root, width=4000, height=1000,
                borderwidth=5, bg='#C0C0C0',cursor="hand2", relief=RIDGE)
canvas1.place(x=550, y=350,width=450,height=200)

showLabel = Label(master=canvas1,textvariable=folder_path, bg=secondary, fg=white,anchor='w', height=2)
showLabel.pack(fill='x', padx=20, pady=60)
showLabel['font'] = showLabelFont

selectButton = Button(showLabel,text="SELECT", command=browse_button, bg=primary, fg=white, height=1)
selectButton.place(relx=0.8, rely=0.2)
selectButton['font'] = browseButtonFont


deleteButton = Button(canvas1,text="CLEAN", command=lambda:check_for_duplicates([folder_path.get()], finished_statement), bg=primary, fg=white, height=1)
deleteButton.place(relx=0.45, rely=0.8)
deleteButton['font'] = deleteButtonFont

finishedLabel = Label(master=canvas1,textvariable=finished_statement, bg=primary, fg=white,anchor='w')
finishedLabel.place(relx=0.5, rely=0.65, anchor=CENTER)
finishedLabel['font'] = finishedFont

# #-------------------------------Button-----------------------------------#
#Button to show image
btn =  Button(root, text = "upload_image", relief=RIDGE,font=('arial',10),bg="#F3F3F3",fg="#222222",cursor="hand2",
command = upload_img)
btn.place(x=250, y = 400)
#pred
btn2 = Button(root, text = "Predict",relief=RIDGE,font=('arial',10),bg="#F3F3F3",fg="#222222",cursor="hand2",
            command = predict_img ) #command = predict_img )
btn2.place(x = 450, y = 400)
#-------------------------------Dataframe-----------------------------------#
#Create dataframe to create tables
f=tk.Frame(width=10,heigh=10)
f.grid(row=30, column=30, columnspan=3)
df=pd.DataFrame({'Disease': ["Cassava Bacterial Blight (CBB)",
                            "Cassava Brown Streak Disease (CBSD)", 
                            "Cassava Green Mo   ttle (CGM)","Cassava Mosaic Disease (CMD)",
                            'Healthy'], 
                'Label' : [0, 1, 2, 3, 4]}) 
table = pt = Table(f, dataframe=df,
        showtoolbar=True, showstatusbar=True)
#showing the table
pt.show()
#-------------------------------CLOCK-----------------------------------#

#google logo
# g_logo = ImageTk.PhotoImage(Image.open('google logo.png'))
# l2 = Label(root, image = g_logo).place(x=60,y=90)

root.mainloop()

Displaying Image.......
C:/Users/Administrator/Desktop/Test/11.JPG
Entered successfully
C:/Users/Administrator/Desktop/Test/11.JPG
Cassava Green Mottle (CGM)


REFERENCE                            
[1] https://github.com/Aashishkumar123/Python-GUI-Project/blob/master/Google%20Engine/google-search-engine.py    
[2]https://github.com/PCTEJA/Tkinter_Ideas/blob/master/adding%20table%20in%20frame.py       

